In [111]:
import torch.nn as nn
import torch
import math
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np


In [149]:
p = torch.rand([32,9, 65440])
y = torch.rand([32,65440])

In [159]:
def focal_loss(p, y, gamma):
    """
    Args:
        p: [32, 9, 65440]
        y = [32, 65440]
    """
    n_classes = p.size(1)
    y = F.one_hot(y.long(),num_classes=n_classes)
    y = y.transpose(1, 2).contiguous()

    alpha = torch.ones([n_classes]) * 1000.0
    alpha[0] = 10
    # alpha.size() = 9

    p = F.softmax(p,dim=1)
    #p: [32, 9, 65440]

    term1 = torch.pow(1 - p,gamma)
    #term1: [32, 9, 65440]

    term2 = torch.log(p)
    #term2: [32, 9, 65440]

    term3 = term1 * y * term2
    
    
    term3 = torch.einsum('a,bcd->bad', -alpha.to(device), term3.to(device))
    #term3: [32, 9, 65440]
    return term3

In [160]:
focal_loss(p,y, gamma=5).sum(dim=1).mean()

tensor(10242.3525, device='cuda:0')

In [152]:
p.long().size()

torch.Size([32, 9, 65440])